# Inference

This notebook is the continuation of `train.ipynb`, so if you have not viewed that notebook and have not run the code from there, please do so before you begin this notebook.

Again, start with installing:

In [ ]:
!pip install numpy pillow scikit-learn torch

and imports:

In [1]:
import json
import joblib
import logging
import numpy as np
import os
import sys
import torch

from PIL import Image
from BART import BART

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d/%m/%Y %H:%M:%S')
logger = logging.getLogger("Batlogger (Inference)")

ModuleNotFoundError: No module named 'BART'

We needed to complete the `infer(path: str, model_path: str, is_dir: bool = True) -> None` method, which restores the label encoder from `train.ipynb`, recreates the model, and performs inference. If we want to run inference on a single image, we can pass it via the command line (not possible in the Jupyter Notebook environment) as:

`python inference.py IMAGE_NAME.jpg`

Alternatively, if all images are located in the `inference_images/` folder (either added manually or automatically - `train.py` also saves 20% of its data into this folder for testing purposes), we can simply run:

`python inference.py`

In both cases, the result is saved in `results.json`:

In [ ]:
def infer(path: str, model_path: str, is_dir: bool = True) -> None:
    label_encoder = joblib.load('label_encoder.joblib')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = BART(num_classes=len(label_encoder.classes_))
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    results: dict[str, str] = {}

    if is_dir:
        files = [(f, os.path.join(path, f)) for f in sorted(os.listdir(path)) if f.lower().endswith('.jpg')]
    else:
        files = [(os.path.basename(path), path)]

    for f, p in files:
        with Image.open(p) as imag:
            img_array = np.array(imag.resize((128, 128)))

        img = torch.tensor(img_array, dtype=torch.float32).permute(2, 0, 1) / 255.0
        # During training DataLoader automatically creates batches to get shape (BS_size, C, H, W),
        # but during inference for a single image we only have (C, H, W), so we need to add batch dimension
        img = img.unsqueeze(0).to(device)

        with torch.no_grad():
            class_idx = torch.max(model(img), 1)[1].item()

        results[f] = label_encoder.inverse_transform([class_idx])[0]

    with open('results.json', 'w') as file:
        json.dump(results, file, indent=4)

    logger.info(f"Saved predictions for {len(results)} images to results.json")

Use the method above like this:

In [ ]:
infer(path='inference_images', model_path='BART-10M.pth', is_dir=True)